In [1]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
def getVariants(y):
    variants = []
    for i in range(0, np.amax(y) + 1):
        variants.append([])

    for i in range(0, y.shape[0]):
        variants[y[i]].append(i)

    return variants

In [3]:
from sklearn.linear_model import LogisticRegression
class OneVsOne(object):
    def __init__(self):
        self.models = []
        
    def fit(self, dataSet, y):
        variants = getVariants(y)
        setosa_y = (dataSet['target'][variants[0]]).astype(int).reshape(50,1)
        versicolor_y = (dataSet['target'][variants[1]]).astype(int).reshape(50,1)
        virginica_y = (dataSet['target'][variants[2]]).astype(int).reshape(50,1)

        setosa_x = dataSet['data'][variants[0]]
        versicolor_x = dataSet['data'][variants[1]]
        virginica_x = dataSet['data'][variants[2]]
        
        modelAB = LogisticRegression(C=10**10)
        modelAC = LogisticRegression(C=10**10)
        modelBC = LogisticRegression(C=10**10)

        tempX = np.concatenate((setosa_x[0:40], versicolor_x[0:40]), axis = 0)
        tempY = np.concatenate((setosa_y[0:40], versicolor_y[0:40]), axis = 0)

        modelAB.fit(tempX, tempY.ravel())

        tempX = np.concatenate((setosa_x[0:40], virginica_x[0:40]), axis = 0)
        tempY = np.concatenate((setosa_y[0:40], virginica_y[0:40]), axis = 0)

        modelAC.fit(tempX, tempY.ravel())

        tempX = np.concatenate((virginica_x[0:40], versicolor_x[0:40]), axis = 0)
        tempY = np.concatenate((virginica_y[0:40], versicolor_y[0:40]), axis = 0)

        modelBC.fit(tempX, tempY.ravel())
        
        self.models = np.array([modelAB, modelAC, modelBC])
                
    def predict(self,  test_datas):
        def _predict(models, test_data):
            test_data = np.reshape(test_data, (1, -1))
            vote = {
    }
            for model in models:
                pred = model.predict(test_data)[0]
                if pred not in vote:
                    vote[pred] = 1
                else:
                    vote[pred] += 1
            vote = sorted(vote.items(), key=lambda x: x[1], reverse=True)
            # print(vote)
            return vote[0][0]
            
        predict = []
        for data in test_datas:
            predict.append(_predict(self.models, data))
        return np.array(predict)

In [4]:
ovo = OneVsOne()

In [5]:
X = iris['data']
y = (iris['target']).astype(int).reshape(150,1)

In [6]:
ovo.fit(iris, y.ravel())

In [7]:
iris['target_names']

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [8]:
variants = getVariants(y.ravel())
setosa_y = (iris['target'][variants[0]]).astype(int).reshape(50,1)
versicolor_y = (iris['target'][variants[1]]).astype(int).reshape(50,1)
virginica_y = (iris['target'][variants[2]]).astype(int).reshape(50,1)

setosa_x = X[variants[0]]
versicolor_x = X[variants[1]]
virginica_x = X[variants[2]]

In [10]:
testData = np.concatenate([setosa_x[40:], versicolor_x[40:], virginica_x[40:]])

ovo.predict(testData)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [13]:
np.concatenate([setosa_y[40:], versicolor_y[40:], virginica_y[40:]]).reshape(30,)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])